In [ ]:
import mne
import re
import os

from statsmodels.stats.multitest import multipletests

import numpy as np
import pandas as pd

from crosspy.core.methods import cplv, cplv_pairwise
from crosspy.preprocessing.seeg.support import clean_montage, drop_monopolar_channels
from crosspy.preprocessing.seeg.seeg_utils import create_reference_mask, get_electrode_distance
from crosspy.preprocessing.signal import preprocess_data_morlet

from bids import BIDSLayout

from joblib import Parallel, delayed

from matplotlib.ticker import ScalarFormatter

import pickle

import tqdm
import glob

import matplotlib.pyplot as plt

import scipy as sp
import scipy.signal
import scipy.io

from collections import defaultdict

import itertools

import seaborn as sns

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
cdict1 = {'red':   ((0.0, 0.0, 0.0),
                   (0.166, 0.43, 0.43),
                   (0.33, 0.7, 0.7),
                   (0.5, 1.0, 1.0),
                   (0.66, 0.8, 0.8),
                   (1.0, 0.6, 0.6)),

         'green': ((0.0, 0.4, 0.4),
                   (0.166, 0.7, 0.7),
                   (0.33, 0.8, 0.8),
                   (0.5, 1.0, 1.0),
                   (0.66, 0.8, 0.8),
                   (1.0,0.0, 0.0)),

         'blue':  ((0.0, 0.8, 0.8),
                   (0.166, 1.0, 1.0),
                   (0.33, 1.0, 1.0),
                   (0.5, 0.4, 0.4),
                   (0.66, 0.0, 0.0),
                   (1.0, 0.0, 0.0))
        }

ripples_cmap = LinearSegmentedColormap('ripplescmap', cdict1)
ripples_blue = ripples_cmap(0)
ripples_red = ripples_cmap(0.99)
ripples_orange = ripples_cmap(0.7)

colors = [ripples_cmap(0), ripples_cmap(0.33), ripples_cmap(0.66), ripples_cmap(0.99)]

In [ ]:
#df = pd.read_csv('../seeg_phases/data/SEEG_redux_BIDS/convert_results.csv', sep=',')
df = pd.read_csv('/home/gabri/localdata/convert_results.csv', sep=';', skiprows=1)
df_num_idx = df.set_index('subject_number')

subject_to_cluster = pd.read_csv('/home/gabri/localdata/Subject_agg_cluster_labels.csv').set_index('subject_number')

In [ ]:
def create_ez_mask(ez_chans):
    res = np.zeros((ez_chans.shape[0], ez_chans.shape[0]), dtype=bool)
    for i in range(ez_chans.shape[0]):
        for j in range(ez_chans.shape[0]):
            res[i,j] = ez_chans[i] == 0 and ez_chans[j] == 0
    
    return res

def create_gmpi_masks(ch_names, meta):
    def _get_type(idx):
        if -0.3 < ch_gmpi[idx] <= 0:
            return 1
        elif 0.5 < ch_gmpi[idx] < 1:
            return 0
        else:
            return -1
    
    n_chans = len(ch_names)
    ch_gmpi = [meta.loc[ch]['GMPI'] for ch in ch_names]
    
    ss_mask = np.zeros((n_chans, n_chans), dtype=bool)
    dd_mask = np.zeros((n_chans, n_chans), dtype=bool)
    
    for i, j in itertools.product(range(n_chans), range(n_chans)):
        i_type = _get_type(i)
        j_type = _get_type(j)
        
        if i_type == 1 and j_type == 1:
            dd_mask[i,j] = True
        elif i_type == 0 and j_type == 0:
            ss_mask[i,j] = True
    
    return dd_mask, ss_mask

def create_gmpi_masks_2(gmpi, shuffle=False):
    
    gmpi[np.isnan(gmpi)] = -200.

    dd_ = np.logical_and(-.3 < gmpi, gmpi < 0)
    ss_ = np.logical_and(.5 < gmpi, gmpi < 1)
    
    if shuffle:
        dd_indices = np.nonzero(dd_)[0]
        ss_indices = np.nonzero(ss_)[0]
        n_dd = dd_indices.shape[0]
        n_ss = ss_indices.shape[0]
        edge_indices = np.concatenate((dd_indices,ss_indices))
        np.random.shuffle(edge_indices)
        dd_mix = edge_indices[:n_dd]
        ss_mix = edge_indices[n_dd:]
        dd_ = np.zeros((dd_.shape),dtype=int)
        ss_ = np.zeros((ss_.shape),dtype=int)

        dd_[dd_mix] = 1
        ss_[ss_mix] = 1
        
    dd_mask = np.dot(dd_[:,np.newaxis],dd_[:,np.newaxis].T)
    ss_mask = np.dot(ss_[:,np.newaxis],ss_[:,np.newaxis].T)
    
    return dd_mask, ss_mask

def get_spectrum_by_bins(data, filter_mask, distance_matrix):
    spectrum = data['cplv_spectrum']
    surrogate_spectrum = data['surrogate_spectrum']
    
    n_bins = len(distance_bins) - 1
    dists_binned = np.digitize(distance_matrix, bins=distance_bins) - 1
    
    plv_binned = np.zeros((n_bins, spectrum.shape[0]))
    sur_plv_binned = np.zeros((n_bins, spectrum.shape[0]))
    iplv_binned = np.zeros((n_bins, spectrum.shape[0]))
    sur_iplv_binned = np.zeros((n_bins, spectrum.shape[0]))
    plv_k = np.zeros((n_bins, spectrum.shape[0]))
    iplv_k = np.zeros((n_bins, spectrum.shape[0]))
    
    for bin_idx in range(n_bins):
        bin_mask = (dists_binned == bin_idx)
        
        mask = np.triu(filter_mask & bin_mask, 1)

        if np.any(mask > 0):
            plv_masked = np.abs(spectrum[:, mask])
            iplv_masked = np.abs(np.imag(spectrum[:, mask]))

            plv_surr_values = np.abs(surrogate_spectrum[:,mask]).mean(axis=1, keepdims=True)*3.52
            iplv_surr_values = np.abs(np.imag(surrogate_spectrum[:,mask])).mean(axis=1, keepdims=True)*3.52

            plv_binned[bin_idx] = plv_masked.mean(axis=1)
            iplv_binned[bin_idx] = iplv_masked.mean(axis=1)
            sur_plv_binned[bin_idx] = np.abs(surrogate_spectrum[:,mask]).mean(axis=1)
            sur_iplv_binned[bin_idx] = np.abs(np.imag(surrogate_spectrum[:,mask])).mean(axis=1)
            
            plv_k[bin_idx] = np.mean(plv_masked >= plv_surr_values, axis=1)
            iplv_k[bin_idx] = np.mean(iplv_masked >= iplv_surr_values, axis=1)
    
    return plv_binned, iplv_binned, plv_k, iplv_k, sur_plv_binned, sur_iplv_binned
    
def is_bipolar(x):
    a, c = x.split('-')
    
    return len(c) > 0

def get_frequencies():
    return np.load('gabriele/frequencies.npy')

def _get_meta(path_subject, file_type):
    return glob.glob(path_subject+'/*'+file_type+'*')[0]

In [ ]:
distance_bins = [0,  32,  45,  60, 137]

## actual routine 

In [ ]:
root_path = os.path.join('/home/gabri/localdata/rest-bids/')
layout = BIDSLayout('/home/gabri/localdata/rest-bids/')

subject_numbers = list()

n_rep = 1000
# 2 layers, 4 distance ranges, 50 frequencies and 68 subjects
plv_layers = np.zeros((2, 4, 50, 68))
plv_layers_sur = np.zeros((2, 4, 50, 68))
plv_layers_k = np.zeros((2, 4, 50, 68))
plv_layers_stats = np.zeros((2, 4, 50, 68,n_rep))

iplv_layers = np.zeros((2, 4, 50, 68))
iplv_layers_sur = np.zeros((2, 4, 50, 68))
iplv_layers_k = np.zeros((2, 4, 50, 68))
iplv_layers_stats = np.zeros((2, 4, 50, 68,n_rep))

for subject in tqdm.tqdm_notebook(layout.get(target='subject', extension='edf', task='rest')): 
    subj_num = subject.entities['subject']
    subj_name = df_num_idx.loc[int(subj_num)]['name']
    subj_idx = int(subj_num)-1
    
    meta_fname  = '/home/gabri/localdata/rest-bids/sub-{}/ses-01/ieeg/sub-{}_meta.csv'.format(subj_num, subj_num)
    res_fname = '/home/gabri/localdata/rest-bids/derivatives/crosspy/plv_spectrum_cw_wake/sub-{}_spectrum_with_lp.pickle'.format(subject.entities['subject'])

    if subj_num in {'36', '57', '18'} or subject_to_cluster.loc[int(subj_num)]['subject_cluster'] == 4:
        print('Excluding {} from analysis'.format(subj_idx))
        continue
    
    if not(os.path.exists(meta_fname)):
        print('Subj {} does not have meta!'.format(subj_idx))
        continue
        
    subject_numbers.append(subj_idx-1)
    
    subj_meta = pd.read_csv(meta_fname).set_index('contact')

    with open(res_fname, 'rb') as f:
        res_data = pickle.load(f)

    ez_tags = subj_meta.EZ.to_numpy()
    nez_tags = (ez_tags  == 0).astype(int)
    nez_mask = np.dot(nez_tags[:,np.newaxis],nez_tags[:,np.newaxis].T)
    nez_mask = nez_mask.astype(bool)
    nez_mask = np.ones(nez_mask.shape, dtype=bool)

    ref_mask = res_data['reference_mask'].astype(bool)
    frequencies = res_data['frequencies']  
    distance_matrix = res_data['electrodes_distance']
    
    gmpi = subj_meta.GMPI.to_numpy()
    dd_mask, ss_mask = create_gmpi_masks_2(gmpi,shuffle=False)
    
    dd_mask = np.triu(np.logical_and.reduce([ref_mask, nez_mask, dd_mask]), 1)
    ss_mask = np.triu(np.logical_and.reduce([ref_mask, nez_mask, ss_mask]), 1)
    
    dd_out = get_spectrum_by_bins(res_data, dd_mask, distance_matrix)
    ss_out = get_spectrum_by_bins(res_data, ss_mask, distance_matrix)
    
    plv_layers[0,:,:,subj_idx] = dd_out[0]
    plv_layers[1,:,:,subj_idx] = ss_out[0]
    
    iplv_layers[0,:,:,subj_idx] = dd_out[1]
    iplv_layers[1,:,:,subj_idx] = ss_out[1]
    
    plv_layers_k[0,:,:,subj_idx] = dd_out[2]
    plv_layers_k[1,:,:,subj_idx] = ss_out[2]
    
    iplv_layers_k[0,:,:,subj_idx] = dd_out[3]
    iplv_layers_k[1,:,:,subj_idx] = ss_out[3]
    
    plv_layers_sur[0,:,:,subj_idx] = dd_out[4]
    plv_layers_sur[1,:,:,subj_idx] = ss_out[4]
    
    iplv_layers_sur[0,:,:,subj_idx] = dd_out[5]
    iplv_layers_sur[1,:,:,subj_idx] = ss_out[5]

    for rep_idx in range(n_rep):
        
        dd_mask, ss_mask = create_gmpi_masks_2(gmpi,shuffle=True)
        dd_mask = np.triu(np.logical_and.reduce([ref_mask, nez_mask, dd_mask]), 1)
        ss_mask = np.triu(np.logical_and.reduce([ref_mask, nez_mask, ss_mask]), 1)
        
        subj_dd_plv_binned,subj_dd_iplv_binned,subj_dd_k_plv_binned,subj_dd_k_iplv_binned,_,_ = get_spectrum_by_bins(res_data, dd_mask, distance_matrix)
        subj_ss_plv_binned, subj_ss_iplv_binned, _, _,_,_ = get_spectrum_by_bins(res_data, ss_mask, distance_matrix)
        plv_layers_stats[0,:,:,subj_idx,rep_idx] = subj_dd_plv_binned
        plv_layers_stats[1,:,:,subj_idx,rep_idx] = subj_ss_plv_binned
        iplv_layers_stats[0,:,:,subj_idx,rep_idx] = subj_dd_iplv_binned
        iplv_layers_stats[1,:,:,subj_idx,rep_idx] = subj_ss_iplv_binned
        
# remove from resulting matrix the empty subjects
empty_subj = np.nonzero(plv_layers[0,1,0,:])

In [ ]:
plv_layers = np.squeeze(plv_layers[:,:,:,empty_subj])
plv_layers_k = np.squeeze(plv_layers_k[:,:,:,empty_subj])
plv_layers_stats = np.squeeze(plv_layers_stats[:,:,:,empty_subj,:])

iplv_layers = np.squeeze(iplv_layers[:,:,:,empty_subj])
iplv_layers_k = np.squeeze(iplv_layers_k[:,:,:,empty_subj])
iplv_layers_stats = np.squeeze(iplv_layers_stats[:,:,:,empty_subj,:])

## Max T stats

In [ ]:
def get_max_T_stats(group_stats, alpha=.05):
    # compute statistics -> absolute diff between groups i.e. layers
    t_stat = np.mean(np.abs(np.diff(group_stats, axis=0)),axis=3)
    
    # get max t_stat along frequency axis
    max_t_stat = np.max(t_stat, axis=2)

    # get percentiles for one tail test since we're using abs diff
    ci = np.percentile(max_t_stat, (1-alpha)*100, axis=2)

    return np.squeeze(ci)

T_obs = np.squeeze(np.mean(np.abs(np.diff(plv_layers, axis=0)),axis=(3)))
max_T = get_max_T_stats(plv_layers_stats, alpha=0.05)

titles = ['vSH', 'Short','Mid','long']
_, axs = plt.subplots(1,4, figsize=(18,5), sharey=True)
for idx,ax in enumerate(axs):
    #ax.semilogx(frequencies, np.squeeze(np.abs(np.diff(plv_layers[:,idx,:,:], axis=0))),'grey')
    ax.semilogx(frequencies, T_obs[idx,:], 'k', linewidth=2)
    ax.semilogx(frequencies, np.tile(max_T[idx],50), 'k--')
    ax.set_xlabel('Freq. [Hz]')
    ax.set_title(titles[idx])
    ax.xaxis.set_major_formatter(ScalarFormatter())

axs[0].set_ylabel('|diff(ss,dd)|')
axs[0].legend(('layer diff','p<.05 _ max T'))

plt.show()

## B-H FDR correction

In [ ]:
def get_pvalue(group_stats, T_obs):
    # compute statistics -> absolute diff between groups i.e. layers
    t_stat = np.squeeze(np.mean(np.abs(np.diff(group_stats, axis=0)),axis=3))
    
    #compute fraction of T_obs > than T_surr
    pval = np.mean(t_stat >= np.tile(T_obs[:,:,np.newaxis],t_stat.shape[-1:]) , axis=2)
    return pval

def bh_correction(pval, alpha=0.05):
    pval_sort_idx = np.argsort(pval)
    pval_sorted = pval[pval_sort_idx]
    p_ranks = np.arange(1,51)
    p_tmp = (p_ranks/50)*alpha
    h = np.zeros((50,),dtype=bool)
    h[:np.max(np.nonzero(pval_sorted <= p_tmp)[0])] = True
    return h[pval_sort_idx]
    
# T_obs = np.squeeze(np.mean(np.abs(np.diff(iplv_layers, axis=0)),axis=(3)))
iplv_layer_diff = np.abs(np.diff(iplv_layers, axis=0).squeeze())
plv_layer_diff = np.abs(np.diff(plv_layers, axis=0).squeeze())

T_obs = np.mean(np.abs(iplv_layer_diff),axis=(2))
pval = get_pvalue(iplv_layers_stats, T_obs)

titles = ['vSH', 'Short','Mid','long']
_, axs = plt.subplots(1,4, figsize=(18,5), sharey=True)
for idx,ax in enumerate(axs):
    ax.semilogx(frequencies, T_obs[idx,:], 'yo', linewidth=2)
    h = multipletests(pval[idx,:],alpha=.05, method='fdr_bh')[0]
    ax.semilogx(frequencies[h],T_obs[idx,h], 'ko')
    ax.set_xlabel('Freq. [Hz]')
    ax.set_title(titles[idx])
    ax.xaxis.set_major_formatter(ScalarFormatter())

axs[0].set_ylabel('|diff(ss,dd)|')
axs[0].legend(('layer diff','p<.05 _ max T'))

plt.show()

In [ ]:
splits = np.load('cov_1.npy')

In [ ]:
iplv_diff_corr = np.zeros((100, 4))
plv_diff_corr = np.zeros((100, 4))

for sidx, s in enumerate(splits):
    cohort_1 = s[:34] + 1
    cohort_2 = s[34:] + 1

    cohort_1 = [s for s in cohort_1 if s != 36 and subject_to_cluster.loc[s]['subject_cluster'] != 4]
    cohort_2 = [s for s in cohort_2 if s != 36 and subject_to_cluster.loc[s]['subject_cluster'] != 4]

    cohort_1_mask = np.in1d(subject_numbers, cohort_1)
    cohort_2_mask = np.in1d(subject_numbers, cohort_2)
    
    c1_iplv = iplv_layer_diff[..., cohort_1_mask].mean(axis=-1)
    c2_iplv = iplv_layer_diff[..., cohort_2_mask].mean(axis=-1)
    
    c1_plv = plv_layer_diff[..., cohort_1_mask].mean(axis=-1)
    c2_plv = plv_layer_diff[..., cohort_2_mask].mean(axis=-1)
    
    iplv_diff_corr[sidx] = np.diag(np.corrcoef(c1_iplv, c2_iplv), 4)
    plv_diff_corr[sidx] = np.diag(np.corrcoef(c1_plv, c2_plv), 4)

In [ ]:
iplv_diff_corr_surr = np.zeros((100, 4))
plv_diff_corr_surr = np.zeros((100, 4))

for sidx, s in enumerate(splits):
    s = s.copy()
    np.random.shuffle(s)
    
    cohort_1 = s[:34] + 1
    cohort_2 = s[34:] + 1

    cohort_1 = [s for s in cohort_1 if s != 36 and subject_to_cluster.loc[s]['subject_cluster'] != 4]
    cohort_2 = [s for s in cohort_2 if s != 36 and subject_to_cluster.loc[s]['subject_cluster'] != 4]

    cohort_1_mask = np.in1d(subject_numbers, cohort_1)
    cohort_2_mask = np.in1d(subject_numbers, cohort_2)
    
    c1_iplv = iplv_layer_diff[..., cohort_1_mask].mean(axis=-1)
    c2_iplv = iplv_layer_diff[..., cohort_2_mask].mean(axis=-1)
    
    c1_plv = plv_layer_diff[..., cohort_1_mask].mean(axis=-1)
    c2_plv = plv_layer_diff[..., cohort_2_mask].mean(axis=-1)
    
    iplv_diff_corr_surr[sidx] = np.diag(np.corrcoef(c1_iplv, c2_iplv), 4)
    plv_diff_corr_surr[sidx] = np.diag(np.corrcoef(c1_plv, c2_plv), 4)

In [ ]:
cohort_1_mask.shape

In [ ]:
fig, axes = plt.subplots(figsize=(10, 12.5), nrows=2)

# _ = axes[0].boxplot(plv_coefficients)
sns.boxplot(data=plv_diff_corr, ax=axes[0])
sns.swarmplot(data=plv_diff_corr, ax=axes[0], palette=colors)

sns.boxplot(data=iplv_diff_corr, ax=axes[1])
sns.swarmplot(data=iplv_diff_corr, ax=axes[1], palette=colors)

for box in axes[0].artists + axes[1].artists:
    box.set_facecolor('white')
    
axes[0].set_title('PLV layer difference', fontsize=24)
axes[1].set_title('iPLV layer difference', fontsize=24)

for ax in axes:
    ax.set_xticklabels(['Very short', 'Short', 'Medium', 'Long'])
    ax.tick_params(labelsize=18)
    ax.set_ylabel('Correlation', fontsize=20)
    ax.set_ylim([0.4, 1])
    
fig.savefig('split_layer_diff_correlation.svg', dpi=300)

In [ ]:
def estimate_mean_with_bootstrap(spectrum, N_rounds=1000):
    all_indices = np.arange(spectrum.shape[0])
        
    res = np.zeros((N_rounds, ) + spectrum.shape[1:])
    for idx in range(N_rounds):
        round_indices = np.random.choice(all_indices, size=all_indices.shape[0])
        res[idx] = spectrum[round_indices].mean(axis=0)
        
    return res

## create figure 4

In [ ]:
def bootstrap_ci(data, n=100, axis=0, percentiles=(2.5,97.5)):
   
    bootstraps = np.ndarray((n,data.shape[0]), dtype = data.dtype)

    for idx in range(n):
        perm_indices = np.random.randint(0,data.shape[1], data.shape[1])
        bootstraps[idx,:] = data[:,perm_indices].mean(axis=1)
    
    ci = np.squeeze(np.percentile(bootstraps, percentiles, keepdims=True, axis=0))

    return ci[0,:], ci[1,:]

fig, axes = plt.subplots(figsize=(25, 18), ncols=4, nrows=4, sharey='row')

group_plv_layers = np.mean(plv_layers,axis=3)
group_iplv_layers = np.mean(iplv_layers,axis=3)
group_plv_k_layers = np.mean(plv_layers_k,axis=3)
group_iplv_k_layers = np.mean(iplv_layers_k,axis=3)

group_plv_sur_layers = np.mean(plv_layers_sur, axis=3)
group_iplv_sur_layers = np.mean(iplv_layers_sur, axis=3)

plv_T_obs = np.squeeze(np.mean(np.abs(np.diff(plv_layers, axis=0)),axis=(3)))
plv_pval = get_pvalue(plv_layers_stats, plv_T_obs)

iplv_T_obs = np.squeeze(np.mean(np.abs(np.diff(iplv_layers, axis=0)),axis=(3)))
iplv_pval = get_pvalue(iplv_layers_stats, iplv_T_obs)
colors = np.array([[0, 146, 146], [146,0,0]])/255

for bin_idx, ax_row in enumerate(axes.T):
    plv_h = multipletests(plv_pval[bin_idx,:],alpha=.05, method='fdr_bh')[0]
    ax_row[0].set_prop_cycle(color=colors)
    ax_row[0].semilogx(frequencies, group_plv_layers[:,bin_idx,:].T, lw=4)
    ax_row[0].fill_between(frequencies, *bootstrap_ci(plv_layers[0,bin_idx,:,:]), alpha=0.25)
    ax_row[0].fill_between(frequencies, *bootstrap_ci(plv_layers[1,bin_idx,:,:]), alpha=0.25)
    ax_row[0].semilogx(frequencies[plv_h], group_plv_layers[:,bin_idx,plv_h].T, 'o', markersize=10 )
    ax_row[0].semilogx(frequencies, group_plv_sur_layers[:,bin_idx,:].T,'--' ,lw=1)
    ax_row[0].set_ylim([0,.2])
    axins = ax_row[0].inset_axes([0.48, 0.45, 0.4, 0.55])
    axins.xaxis.set_major_formatter(ScalarFormatter())
    axins.set_prop_cycle(color=colors)
    axins.semilogx(frequencies[-18:], group_plv_layers[:,bin_idx,-18:].T, lw=4)
    axins.fill_between(frequencies[-18:], *bootstrap_ci(plv_layers[0,bin_idx,-18:,:]), alpha=0.25)
    axins.fill_between(frequencies[-18:], *bootstrap_ci(plv_layers[1,bin_idx,-18:,:]), alpha=0.25)

    iplv_h = multipletests(iplv_pval[bin_idx,:],alpha=.05, method='fdr_bh')[0]
    ax_row[1].set_prop_cycle(color=colors)
    ax_row[1].semilogx(frequencies, group_iplv_layers[:,bin_idx,:].T, lw=4)
    ax_row[1].fill_between(frequencies, *bootstrap_ci(iplv_layers[0,bin_idx,:,:]), alpha=0.25)
    ax_row[1].fill_between(frequencies, *bootstrap_ci(iplv_layers[1,bin_idx,:,:]), alpha=0.25)
    ax_row[1].semilogx(frequencies[iplv_h], group_iplv_layers[:,bin_idx,iplv_h].T, 'o', markersize=10)
    ax_row[1].semilogx(frequencies, group_iplv_sur_layers[:,bin_idx,:].T,'--', lw=1)
    ax_row[1].set_ylim([0, .08])
    ax_row[1].set_ylim([0, .08])
    axins = ax_row[1].inset_axes([0.48, 0.45, 0.4, 0.55])
    axins.set_prop_cycle(color=colors)
    axins.xaxis.set_major_formatter(ScalarFormatter())
    axins.semilogx(frequencies[-18:], group_iplv_layers[:,bin_idx,-18:].T, lw=4)
    axins.fill_between(frequencies[-18:], *bootstrap_ci(iplv_layers[0,bin_idx,-18:,:]), alpha=0.25)
    axins.fill_between(frequencies[-18:], *bootstrap_ci(iplv_layers[1,bin_idx,-18:,:]), alpha=0.25)
    
    ax_row[2].set_prop_cycle(color=colors)
    ax_row[2].semilogx(frequencies,group_plv_k_layers[:,bin_idx,:].T, lw=4)
    ax_row[2].fill_between(frequencies, *bootstrap_ci(plv_layers_k[0,bin_idx,:,:]), alpha=0.25)
    ax_row[2].fill_between(frequencies, *bootstrap_ci(plv_layers_k[1,bin_idx,:,:]), alpha=0.25)
    ax_row[2].set_ylim([0, 1])

    ax_row[3].set_prop_cycle(color=colors)
    ax_row[3].semilogx(frequencies,group_iplv_k_layers[:,bin_idx,:].T, lw=4)
    ax_row[3].fill_between(frequencies, *bootstrap_ci(iplv_layers_k[0,bin_idx,:,:]), alpha=0.25)
    ax_row[3].fill_between(frequencies, *bootstrap_ci(iplv_layers_k[1,bin_idx,:,:]), alpha=0.25)
    ax_row[3].set_ylim([0, 1])

for ax in axes.flatten():
    ax.tick_params(labelsize=18)
    ax.set_xlabel('Frequency [Hz]', fontsize=24)
    ax.xaxis.set_major_formatter(ScalarFormatter())
    ax.set_xlim([frequencies.min(),frequencies.max()])
    
for ax, name in zip(axes[0], ['Very-short [<32 mm]', 'Short [32 < x <45]', 'Medium [45 < x < 60]', 'Long [60 < x < 137]']):
    ax.set_title(name, fontsize=24)
    ax.legend(('dd','ss'), loc='lower left')

fig.tight_layout()
plt.savefig('/home/gabri/localdata/dPLV/Fig5.pdf', format='pdf')


# Split cohort reliability

In [ ]:
splits = np.genfromtxt('/home/gabri/localdata/rest-bids/At_Least_1_Subj_per_Schaefer-Edge_BestSplit.csv', delimiter=',')
n_splits, n_subjects = splits.shape

idx_split = np.random.randint(0, 100, 1)   

# for each splits get coh1 and coh2 subject indices
coh1_indices = splits[idx_split,:32].astype(int).squeeze()
coh2_indices = splits[idx_split,33:].astype(int).squeeze()

    

In [ ]:
fig, axs = plt.subplots(2,4, figsize=(20,5), sharey='row')

plv1 = np.nanmean(plv_layers[...,coh1_indices],axis=-1)
plv2 = np.nanmean(plv_layers[...,coh2_indices],axis=-1)

for idx, ax in enumerate(axs.T):
    ax[0].set_prop_cycle(color=colors)
    ax[1].set_prop_cycle(color=colors)
    
    ax[0].semilogx(frequencies, plv1[:,idx,:].T)
    for jdx in range(2):
        ax[0].fill_between(frequencies, *bootstrap_ci(plv_layers[jdx,idx,:,coh1_indices].T), alpha=.2)
    ax[1].semilogx(frequencies, plv2[:,idx,:].T)
    
    for jdx in range(2):
        ax[1].fill_between(frequencies, *bootstrap_ci(plv_layers[jdx,idx,:,coh2_indices].T), alpha=.2)
        
        
for ax in axs.flatten():
    ax.tick_params(labelsize=8)
    ax.set_xlabel('Frequency [Hz]', fontsize=8)
    ax.xaxis.set_major_formatter(ScalarFormatter())
    ax.set_xlim([frequencies.min(),frequencies.max()])
    
fig.tight_layout()
fig.savefig('/home/gabri/Fig.S5_split_layers.svg', dpi=300)